In [1]:
import tensorflow as tf
import numpy as np

In [2]:
SEQUENCE_LENGTH = 50
NUM_CLASSES = 10
VOCAB_SIZE = 2000
EMBEDDING_SIZE = 100
LAMBDA = 0.01

In [3]:
# Placeholders for inputs and ouputs
with tf.variable_scope('inputs'):
    x = tf.placeholder(tf.int32, [None, SEQUENCE_LENGTH], name='x')
    y = tf.placeholder(tf.float32, [None, NUM_CLASSES], name='y')
    dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')

In [4]:
# Keeping track of l2 regularization
l2_reg = tf.constant(0.0)

In [5]:
# Embedding layer
with tf.variable_scope('embedding'):
    W = tf.get_variable('W', initializer=tf.random_uniform([VOCAB_SIZE, EMBEDDING_SIZE], -1, 1))
    embedded_chars = tf.nn.embedding_lookup(W, x)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)

In [6]:
FILTER_SIZES = [3,4,5,6,7,8,9,10]
NUM_FILTERS = 5

In [7]:
# Create a convolution + max-pool layer for each filter size
pooled_outputs = []
for i, filter_size in enumerate(FILTER_SIZES):
    with tf.variable_scope('conv-maxpool-%s' % filter_size):
        filter_shape = [filter_size, EMBEDDING_SIZE, 1, NUM_FILTERS]
        W = tf.get_variable('W', initializer=tf.truncated_normal(filter_shape, stddev=0.1))
        b = tf.get_variable('b', initializer=tf.constant(0.1, shape=[NUM_FILTERS]))
        conv = tf.nn.conv2d(embedded_chars_expanded, W, strides=[1,1,1,1], padding='VALID', name='conv')
        activations = tf.nn.relu(tf.nn.bias_add(conv, b), name='activation')
        pooled = tf.nn.max_pool(activations, ksize=[1,SEQUENCE_LENGTH-filter_size+1,1,1], strides=[1,1,1,1], 
                               padding='VALID', name='pool')
        pooled_outputs.append(pooled)

In [8]:
# Combine all the pooled features
num_filters_total = NUM_FILTERS * len(FILTER_SIZES)
h_pool = tf.reshape(tf.concat(pooled_outputs, 3), [-1, num_filters_total])

In [9]:
# Add dropout
with tf.variable_scope('dropout'):
    h_drop = tf.nn.dropout(h_pool, dropout_keep_prob)

In [10]:
with tf.variable_scope('outputs'):
    W = tf.get_variable('W', shape=[num_filters_total, NUM_CLASSES], 
                        initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable('b', initializer=tf.constant(0.1, shape=[NUM_CLASSES]))
    l2_reg += tf.nn.l2_loss(W)
    l2_reg += tf.nn.l2_loss(b)
    scores = tf.nn.xw_plus_b(h_drop, W, b, name='scores')
    pred = tf.argmax(scores, 1, name='predictions')

In [11]:
with tf.variable_scope('performance'):
    loss_func = tf.nn.softmax_cross_entropy_with_logits(logits=scores, labels=y)
    loss = tf.reduce_mean(loss_func) + LAMBDA * l2_reg
    correct = tf.equal(pred, tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred, 'float'), name='accuracy')